In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import calinski_harabasz_score

In [ ]:
file_path = r"C:\Users\mipo\OneDrive - Hillyard, Inc\Desktop\GitHub\hillyard_routing_analysis\Resources\Lancaster.csv"
data = pd.read_csv(file_path)

In [ ]:
selected_columns = ['Latitude', 'Longitude']
X = data[selected_columns]
#, 'Total Orders', 'Total Deliveries', 'Avg. Revenue' - removed from range

In [ ]:
X.fillna(0, inplace=True)

In [ ]:
#determine the range of the amount of clusters to analyze
k_values = range(2,8)
ch_scores = []

for k in k_values:
    kmeans =  KMeans(n_clusters=k)
    kmeans.fit(X)
    labels = kmeans.labels_
    ch_score = calinski_harabasz_score(X, labels)
    ch_scores.append(ch_score)

In [ ]:
plt.figure(figsize=(20, 12))
plt.plot(k_values, ch_scores, marker='o')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Calinski-Harabasz Score')
plt.title('Calinski-Harabasz Score for Different Values of k')
plt.xticks(k_values)
plt.grid(True)
plt.show()

In [ ]:
k = k_values[np.argmax(ch_scores)]
k

In [ ]:
kmeans = KMeans(n_clusters=k)
kmeans.fit(X)
labels = kmeans.labels_
centroids = kmeans.cluster_centers_

In [ ]:
plt.scatter(X.iloc[:, 0], X.iloc[:, 1], c=labels, cmap='viridis')
plt.scatter(centroids[:, 0], centroids[:, 1], c='red', marker='x')
plt.title('K-means Clustering')
plt.xlabel('Latitude')
plt.ylabel('Longitude')
plt.show()

In [ ]:
print("Cluster Centroids:")
for i, centroid in enumerate(centroids):
    print(f"Cluster {i+1}: Latitude = {centroid[0]}, Longitude = {centroid[1]}")

In [ ]:
cluster_centroids_df = pd.DataFrame(centroids, columns=X.columns)
cluster_centroids_df

In [ ]:
cluster_counts = pd.Series(labels).value_counts().sort_index()
cluster_centroids_df['Location Count'] = cluster_counts.values
cluster_centroids_df

In [ ]:
data['Cluster'] = labels
data

In [ ]:
cluster_centroids_df.to_csv("1430_cluster_centroids_2.csv", index=False)
data.to_csv("1430_custered_stops_2.csv", index=False)